<img src="../files/misc/logo.png" width=300/>
<h1 style="color:#872325"> Webscrapping </h1>

De acuerdo a [Wikipedia](https://es.wikipedia.org/wiki/Web_scraping), *Web Scrapping*, 

> es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la World Wide Web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

Para fines prácticos, realizamos un web scrapping cuándo no contamos con alguna manera de acceder a información estructurada de datos por analizar (.csv, .json, base de datos, API). 

---

## HTML
Al hacer webscrapping, dependemos de la estructura de la página web para poder acceder a la información que deseamos obtener. La estructura de la página web se define mediante un lenguaje llamado `HTML` (Hypertext Markup Language) el cual se compone por medio de *tags*.

```HTML
<tag>
</tag>
```

Un *tag* define una sección de la página. Cada *tag* tiene una propiedad. Un conjunto de estos tags definen el cuerpo de la página.


### Una primera página web
```HTML
<!DOCTYPE html>
<html>
<body>

<h1>Data Science and Machine Learning</h1>

</body>
</html>
```

Para nuestros fines, un *tag* representará una sección dentro de la página la cuál nos gustaría explorar o extraer.

---

## CSS

Un *Cascading Style Sheet* (CSS), es un archivo que viene acompañado de un html y define el estilo de la página.

```CSS
selector {
    property: property-value;
}
```

Cada *selector* contiene diferentes [propiedades](http://web.simmons.edu/~grabiner/comm244/weekthree/css-basic-properties.html) de acuerdo al *tag*, clase o *id* que le corresponda.

Para fines de webscrapping, un los *tags*, *ids* y *clases* ofrencen estrucura dentro de una página.

* Tags: estructura general
* Clase: secciones genéricas dentro de una página
* *id*: Secciones especifícas

---

## Javascript y el DOM

La mayoría de las páginas que visitamos hoy en día son interactivas. La interactividad de un una página web es proporcionada por un lenguaje de programación llamado `javascript`.

La manera en la que Javascript asigna u obtiene elementos de una página se conoce como el *Document Object Notation* (DOM).

Para fines de webscrapping, esto es importante puesto que en ocasiones, una página es vacía hasta que un programa de Javascript se ejecuta para arrojar información.

## Un primer webscrap

En este módulo estaremos ocupando la librería [requests_html](https://github.com/psf/requests-html) para hacer webscrapping.

In [8]:
from requests_html import HTML    

In [29]:
with open("../files/lec04/ex03.html") as f:
    webp = f.read()

html = HTML(html=webp)
print(html.html)

<!DOCTYPE html>
<html>
<head>
    <link href="style2.css" rel="stylesheet"/>
</head>

<body>

    <h1>Data Science and Machine Learning</h1>

    <div class="my_class">
        <p> By <a href="https://bit.ly/34EdyH2">Analysic Nabla</a></p>
        <p> At the <a href="https://bit.ly/2qDvao4">ITAM</a></p>
    </div>

</body>
</html>



In [30]:
# Extrayendo todos los links de la página
html.links

{'https://bit.ly/2qDvao4', 'https://bit.ly/34EdyH2'}

In [35]:
# Extrayendo el primer encabezado de la página
html.find("H1", first=True).text

'Data Science and Machine Learning'

In [41]:
# Buscando todos los elementos <p> (paragrapshs)
html.find("p")

[<Element 'p' >, <Element 'p' >]

In [50]:
# Extrayendo una sección dentro de un div
html.find("div", first=True)

<Element 'div' class=('my_class',)>

### Un segundo ejemplo

In [42]:
with open("../files/lec04/ex04.html") as f:
    webp = f.read()

html = HTML(html=webp)
print(html.html)

<!DOCTYPE html>
<html>
<head>
    <link href="style2.css" rel="stylesheet"/>
    <script>
        function sendGreeting() {
            document.getElementById("greeting").innerHTML = "Aprendamos a hacer webscrapping!"
        }
        

    </script>
</head>

<body onload="sendGreeting()">
    <h1>Data Science and Machine Learning</h1>

    <div class="my_class">
        <p>Por Analysic Nabla</p>
        <p>En el ITAM</p>
        <p id="greeting"></p>
    </div>

</body>
</html>



In [52]:
# ¿Qué notamos en este ejemplo que no concuerde
# con lo que esperaríamos ver en la página?
for item in html.find("p"):
    print(item.text)

Por Analysic Nabla
En el ITAM



In [55]:
html.find("p[id='greeting']", first=True)

<Element 'p' id='greeting'>

<h1 style="color:#872325">HTTP Requests</h1>

Las páginas que visitamos día a día viven en un servidor y no en nuestra computadora. Al abrir nuestro navegador y entrar a una página, el navegador solicita la información de la página al servidor por medio de un proceso de intercambio de información conocido como HTTP.

> *Hypertext Transfer Protocol* (HTTP) es un proceso mediante el cual se hace un intercambio de información entre un cliente (un navegador) y un servidor por medio de mensajes.

![server](https://mdn.mozillademos.org/files/13677/Fetching_a_page.png)
**Fuente**x: https://developer.mozilla.org

Dependiendo del tipo de mensaje que le mandemos a un servidor, este procesará la información enviada desde el cliente (nosotros) de diferentes maneras. Para fines de webscrapping, los métodos más comúnes de comunicarte con un servidor son:

* `GET`: Solicitia un archivo determinado al servidor. Un *GET Request* se debe usar exclusivamente para adquirir información.
    * Obtener información de una página web
    * Cargar una imágen

* `POST`: Envía datos al servidor para ser procesados
    * Dejar tu número celular en una página
    * Loggearte
    * Dejar un comentario en un blog post

En Python, podemos hacer conexiones a una página por medio de la librería `requests`

In [57]:
import requests

In [63]:
r = requests.get("https://nabla.mx")
r

<Response [200]>

Posteriormente tendríamos que pasarlo a una instancia de la clase `HTML` para poder analizar la página. Para evitar esto, podemos hacer uso de la clase `HTMLSession` que nos permite hacer llamadas al servidor y acceso al html que nos regrese.

In [59]:
from requests_html import HTMLSession

In [66]:
with HTMLSession() as sess:
    r = sess.get("https://nabla.mx")
r

<Response [200]>

In [72]:
r.html.links

{'https://analysic-nabla.github.io/',
 'https://melectrico.nabla.mx/',
 'https://twitter.com/AnalysicNabla',
 'https://www.facebook.com/Analysic-Nabla-326424374613601/',
 'https://www.instagram.com/analysicnabla/?hl=es-la',
 'https://www.linkedin.com/company/analysic-nabla/',
 'https://www.nabla.mx',
 'https://www.nabla.mx/acerca-de',
 'https://www.nabla.mx/contacto',
 'https://www.nabla.mx/novedades',
 'https://www.nabla.mx/proyectos',
 'https://www.nabla.mx/servicios'}

<h1 style="color:#872325">Webscrapping</h1>

<h2 style="color:teal">Ejemplo: Arxiv ML Papers (I)</h2>

Consigue el nombre y el link de los último 25 papers en Machine Learning publicados en Arxiv y guárdalos dentro de un archivo `csv`.

In [87]:
from requests_html import HTMLSession
import pandas as pd

In [77]:
url = "https://arxiv.org/list/stat.ML/recent"
with HTMLSession() as sess:
    r = sess.get(url)
r

<Response [200]>

In [ ]:
divpage = r.html.find("div[id='dlpage']", first=True)

In [83]:
divpage.find("h1", first=True).text

'Machine Learning'

In [85]:
papers_links = divpage.find("dt")
len(papers_links)

25

In [86]:
papers_titles = divpage.find("dd")
len(papers_titles)

25

In [119]:
values = []
for link_secc, title_secc in zip(papers_links, papers_titles):
    title = title_secc.find("div[class='list-title mathjax']", first=True).text
    
    abstract_url = link_secc.find("a[title='Abstract']", first=True).absolute_links
    abstract_url, *_ = abstract_url
    
    values.append([title, abstract_url])

In [122]:
df_values = pd.DataFrame(values, columns=["title", "url"])
df_values.head()

,title,url
0,Title: Integrating Markov processes with struc...,https://arxiv.org/abs/1911.02175
1,Title: Information-Theoretic Generalization Bo...,https://arxiv.org/abs/1911.02151
2,Title: Designing over uncertain outcomes with ...,https://arxiv.org/abs/1911.02106
3,Title: An Alternative Probabilistic Interpreta...,https://arxiv.org/abs/1911.02088
4,Title: MLPerf Inference Benchmark,https://arxiv.org/abs/1911.02549


In [126]:
# Depuramos la información que no necesitamos
df_values = df_values.assign(title=df_values.title.str.lstrip("Title: "))
df_values.head()

,title,url
0,Integrating Markov processes with structural c...,https://arxiv.org/abs/1911.02175
1,Information-Theoretic Generalization Bounds fo...,https://arxiv.org/abs/1911.02151
2,Designing over uncertain outcomes with stochas...,https://arxiv.org/abs/1911.02106
3,An Alternative Probabilistic Interpretation of...,https://arxiv.org/abs/1911.02088
4,MLPerf Inference Benchmark,https://arxiv.org/abs/1911.02549


In [127]:
# df_values.to_csv("ML-arxiv.csv", index=False)

## Queries al sistema

En ocasiones un *GET method* puede venir acompañado con parámetros obligatorios u opcionales para el sistema.

* Se denota una sección de parámetros de consulta por medio de `?`.
* cada par parametro-valor se denota `param=val`
* se separa cada query por medio de `&`

```
http://some-page.com?key1=val1&key2=val2&..&keyn=valn
```

In [129]:
url = "https://arxiv.org/list/stat.ML/recent?show=250"
with HTMLSession() as sess:
    r = sess.get(url)
r

<Response [200]>

In [132]:
papers_titles = r.html.find("div[id='dlpage']", first=True).find("dd")
len(papers_titles)

250

### Una manera más limpia de mandar queries

In [133]:
url = "https://arxiv.org/list/stat.ML/recent"
data = {
    "show": 250
}
with HTMLSession() as sess:
    r = sess.get(url, params=data)
r

<Response [200]>

In [134]:
papers_titles = r.html.find("div[id='dlpage']", first=True).find("dd")
len(papers_titles)

250

<h1 style="color:#872325">APIs</h1>

En ocasiones no es necesario acceder a un HTML a fin de obtener la información deseada.

In [135]:
url = "https://www.alphavantage.co"

<h2 style="color:crimson">Ejercicios</h2>

1. Repite el ejemplo *Arxiv ML Papers (I)*, agrega dos nuevas columnas al DataFrame final: `Abstract`, con el resumen de cada paper; `Submitted`, con la fecha en la que fue agregado al sistema. **Hint**: Ambos elementos se encuentran dentro de la *url* encontrada.

## References


* https://developer.mozilla.org/en-US/docs/Web
* https://www.w3schools.com/tags/ref_httpmethods.asp